In [9]:
import json
import os
import re
import torch
from torch import Tensor
import random 


In [36]:
ANS_RE = re.compile(r"(.*?)####\s*(-?[0-9.,]+)")

INVALID_ANS = "[invalid]"

In [37]:
def extract_answer(completion):
    # Search for the pattern capturing text before and after '####'
    completion = re.sub(r"<<.*?>>", "", completion)
    # Search for the pattern capturing text before and after '####'
    match = ANS_RE.search(completion)
    if match:
        # Extract rationale and answer, removing unnecessary spaces and commas
        rationale = match.group(1).strip()
        answer = match.group(2).strip().replace(",", "")  # Remove commas from numbers, if any
        return rationale, answer
    return INVALID_ANS  # Return invalid if no pattern matches

In [38]:
x = {'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}

In [39]:
x['answer']

'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'

In [40]:
extract_answer(x['answer'])

('', '72')

In [42]:
ANS_RE = re.compile(r"#### (\-?[0-9\.\,]+)")
INVALID_ANS = "[invalid]"


def extract_answer(completion):
    match = ANS_RE.search(completion)
    if match:
        match_str = match.group(1).strip()
        match_str = match_str.replace(",", "")
        return match_str
    else:
        return INVALID_ANS


In [44]:
extract_answer(x['answer'])

'72'

In [48]:
import re

# Updated regex to capture text before and after '####'
ANS_RE = re.compile(r"(.*?)####\s*(-?[0-9.,]+)", re.DOTALL)  # Ensure re.DOTALL is used if multiline handling is necessary
INVALID_ANS = "[invalid]"

def extract_answer(completion):
    print("Original completion:", completion)  # Debug print
    # Remove placeholder text enclosed in <<...>>
    completion = re.sub(r"<<.*?>>", "", completion)
    
    # Search for the pattern capturing text before and after '####'
    match = ANS_RE.search(completion)
    if match:
        # Extract rationale and answer, removing unnecessary spaces and commas
        rationale = match.group(1).strip()
        answer = match.group(2).strip().replace(",", "")  # Remove commas from numbers, if any
        return rationale, answer
    return INVALID_ANS  # Return invalid if no pattern matches

# Test data
x = {
    'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
    'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'
}

# Call the function and print the result
rationale, answer = extract_answer(x['answer'])
print("Rationale:", rationale)
print("Answer:", answer)


Original completion: Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72
Rationale: Natalia sold 48/2 = 24 clips in May.
Natalia sold 48+24 = 72 clips altogether in April and May.
Answer: 72


In [49]:
rationale

'Natalia sold 48/2 = 24 clips in May.\nNatalia sold 48+24 = 72 clips altogether in April and May.'

In [14]:
# %pip install -U sentence-transformers

# from sentence_transformers import SentenceTransformer, util
# sentences = ["I'm happy", "I'm full of happiness"]

# sentences = ['This is a test; it includes punctuation.','This is a test it does not include punctuation']
sentences = ['This is a test that checks different length sentences', 'I love my cat']


# From https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/util.py
def cos_sim(a: Tensor, b: Tensor) -> Tensor:
    """
    Computes the cosine similarity cos_sim(a[i], b[j]) for all i and j.

    :return: Matrix with res[i][j]  = cos_sim(a[i], b[j])
    """
    if not isinstance(a, torch.Tensor):
        a = torch.tensor(a)

    if not isinstance(b, torch.Tensor):
        b = torch.tensor(b)

    if len(a.shape) == 1:
        a = a.unsqueeze(0)

    if len(b.shape) == 1:
        b = b.unsqueeze(0)

    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    return torch.mm(a_norm, b_norm.transpose(0, 1))


model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#Compute embedding for both lists
embedding_1= model.encode(sentences[0], convert_to_tensor=True)
embedding_2 = model.encode(sentences[1], convert_to_tensor=True)

print(cos_sim(embedding_1, embedding_2).item())



0.11381016671657562


In [ ]:
0.6002566814422607
0.949130654335022
0.11381016671657562